## import 

In [1]:
import torch
import torchvision
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
import pandas as pd
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset
import torch
import torch.nn as nn
import numpy as np
from torch.optim import optimizer
from sklearn import datasets
import matplotlib.pyplot as plt

## read csv and input normalization

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv("/content/drive/MyDrive/code/python/projects/ADL/ADL_(IM5062)/111ntu-homework1/training_data.csv")

In [4]:
df

,AMB_TEMP,CH4,CO,NMHC,NO,NO2,NOx,PM10,PM2.5,RH,SO2
0,16.0,2.0,0.53,0.18,-0.30,16.0,16.0,106.0,105.4,77.0,2.6
1,16.0,2.0,0.56,0.19,0.00,23.0,23.0,106.0,97.6,77.0,3.8
2,16.0,2.0,0.56,0.20,0.10,23.0,23.0,107.0,97.6,79.0,3.6
3,16.0,2.1,0.61,0.22,1.72,19.2,20.9,108.0,98.9,80.0,3.7
4,16.0,2.1,0.67,0.19,-0.40,19.0,18.0,105.0,110.6,80.0,3.6
...,...,...,...,...,...,...,...,...,...,...,...
8779,20.0,1.9,0.30,0.09,1.30,12.0,13.0,63.0,23.5,86.0,1.7
8780,20.0,1.9,0.28,0.09,1.40,13.0,14.0,55.0,26.1,87.0,1.6
8781,20.0,2.0,0.33,0.11,1.30,14.0,16.0,50.0,28.7,89.0,1.5
8782,19.0,2.0,0.37,0.12,1.30,19.0,20.0,52.0,19.6,89.0,1.6


In [5]:
df['AMB_TEMP'].mean()

25.256894353369763

In [6]:
AMB_TEMP_mean = df['AMB_TEMP'].mean()
CH4_mean = df['CH4'].mean()
CO_mean = df['CO'].mean()
NMHC_mean = df['NMHC'].mean()
NO_mean = df['NO'].mean()
NO2_mean = df['NO2'].mean()
NOx_mean = df['NOx'].mean()
PM10_mean = df['PM10'].mean()
RH_mean = df['RH'].mean()
SO2_mean = df['SO2'].mean()
PM2_5_mean = df['PM2.5'].mean()

# std
AMB_TEMP_std = df['AMB_TEMP'].std()
CH4_std = df['CH4'].std()
CO_std = df['CO'].std()
NMHC_std = df['NMHC'].std()
NO_std = df['NO'].std()
NO2_std = df['NO2'].std()
NOx_std = df['NOx'].std()
PM10_std = df['PM10'].std()
RH_std = df['RH'].std()
SO2_std = df['SO2'].std()
PM2_5_std = df['PM2.5'].std()

In [7]:
df['AMB_TEMP'] = (df['AMB_TEMP'] -AMB_TEMP_mean)/AMB_TEMP_std
df['CH4'] = (df['CH4'] -CH4_mean)/CH4_std
df['CO'] = (df['CO'] -CO_mean)/CO_std
df['NMHC'] = (df['NMHC'] -NMHC_mean)/NMHC_std
df['NO'] = (df['NO'] -NO_mean)/NO_std
df['NO2'] = (df['NO2'] -NO2_mean)/NO2_std
df['NOx'] = (df['NOx'] -NOx_mean)/NOx_std
df['PM10'] = (df['PM10'] -PM10_mean)/PM10_std
df['RH'] = (df['RH'] -RH_mean)/RH_std
df['SO2'] = (df['SO2'] -SO2_mean)/SO2_std
df['PM2.5'] = (df['PM2.5'] -PM2_5_mean)/PM2_5_std

In [8]:
target = df['PM2.5'] 

In [9]:
target

0       2.505148
1       2.212519
2       2.212519
3       2.261291
4       2.700233
          ...   
8779   -0.567449
8780   -0.469906
8781   -0.372364
8782   -0.713763
8783   -1.006392
Name: PM2.5, Length: 8784, dtype: float64

In [10]:
target = torch.tensor(target.to_numpy().astype(np.float32))
target = target.view(target.shape[0], 1)

In [11]:
target

tensor([[ 2.5051],
        [ 2.2125],
        [ 2.2125],
        ...,
        [-0.3724],
        [-0.7138],
        [-1.0064]])

In [12]:
df=df.drop(columns=['PM2.5'])

In [13]:
#df

In [14]:
feature = torch.from_numpy(df.to_numpy().astype(np.float32))

In [15]:
feature 

tensor([[-1.7025,  1.1585,  0.9756,  ...,  1.7915,  0.0790, -0.1622],
        [-1.7025,  1.1585,  1.1483,  ...,  1.7915,  0.0790,  0.6857],
        [-1.7025,  1.1585,  1.1483,  ...,  1.8252,  0.2728,  0.5444],
        ...,
        [-0.9669,  1.1585, -0.1762,  ..., -0.0957,  1.2416, -0.9395],
        [-1.1508,  1.1585,  0.0542,  ..., -0.0283,  1.2416, -0.8688],
        [-1.1508,  1.1585,  0.2270,  ...,  0.0391,  1.3385, -0.7275]])

In [16]:
df1 = pd.read_csv("/content/drive/MyDrive/code/python/projects/ADL/ADL_(IM5062)/111ntu-homework1/testing_data.csv")

In [17]:
df1

,AMB_TEMP,CH4,CO,NMHC,NO,NO2,NOx,PM10,RH,SO2
0,19.0,2.1,0.40,0.19,1.8,20.0,22.0,57.0,91.0,2.1
1,18.0,2.2,0.46,0.18,3.9,19.0,23.0,65.0,92.0,2.6
2,18.0,2.3,0.46,0.17,5.3,19.0,25.0,61.0,92.0,2.6
3,18.0,2.3,0.46,0.19,14.0,22.0,37.0,58.0,93.0,2.5
4,18.0,2.2,0.37,0.16,4.1,17.0,21.0,41.0,93.0,2.0
...,...,...,...,...,...,...,...,...,...,...
8755,17.0,2.2,0.85,0.17,0.9,16.0,17.0,177.0,69.0,2.7
8756,17.0,2.2,0.82,0.17,0.9,18.0,18.0,143.0,70.0,2.3
8757,17.0,2.1,0.78,0.16,1.1,15.0,17.0,110.0,72.0,2.3
8758,16.0,2.1,0.73,0.14,1.2,17.0,18.0,96.0,73.0,2.4


In [18]:
df1['AMB_TEMP'] = (df1['AMB_TEMP'] -AMB_TEMP_mean)/AMB_TEMP_std
df1['CH4'] = (df1['CH4'] -CH4_mean)/CH4_std
df1['CO'] = (df1['CO'] -CO_mean)/CO_std
df1['NMHC'] = (df1['NMHC'] -NMHC_mean)/NMHC_std
df1['NO'] = (df1['NO'] -NO_mean)/NO_std
df1['NO2'] = (df1['NO2'] -NO2_mean)/NO2_std
df1['NOx'] = (df1['NOx'] -NOx_mean)/NOx_std
df1['PM10'] = (df1['PM10'] -PM10_mean)/PM10_std
df1['RH'] = (df1['RH'] -RH_mean)/RH_std
df1['SO2'] = (df1['SO2'] -SO2_mean)/SO2_std


In [19]:
#df1

In [20]:
testing = torch.from_numpy(df1.to_numpy().astype(np.float32))

In [21]:
#testing

## Model

In [22]:
n_samples, n_features = feature.shape

# 1) model
class LinearRegression(nn.Module):
    
    def __init__(self, input_dim, hidden_size1, hidden_size2, hidden_size3, hidden_size4, output_dim):        #
        super(LinearRegression, self).__init__()
        
        # define layers
        self.linear1 = nn.Linear(input_dim, hidden_size1)
        self.linear2 = nn.Linear(hidden_size1, hidden_size2)
        self.linear3 = nn.Linear(hidden_size2, hidden_size3)
        self.linear4 = nn.Linear(hidden_size3, hidden_size4)

        self.linear11 = nn.Linear(hidden_size4, output_dim)

    def forward(self, x):
        out = self.linear1(x)
        out = F.relu(out)

        out = self.linear2(out)
        out = F.relu(out)

        out = self.linear3(out)
        out = F.relu(out)

        out = self.linear4(out)
        out = F.relu(out)

        out = self.linear11(out)

        return out

## loss and optimizer

In [23]:
model = LinearRegression(n_features, 60, 60, 60, 60 , 1)

# 2) loss and optimizer
learning_rate = 0.01
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


## training and mini batch

In [24]:
# 3) training loop
epochs = 50
batch_size = 24
for epoch in range(epochs):
  for i in range(int(len(feature)/batch_size)):
      # forward pass and loss
      y_predicted = model(feature[24*i:24*(i+1)])
      loss = criterion(y_predicted, target[24*i:24*(i+1)])

      # backward pass
      loss.backward()

      # update
      optimizer.step()
      
      # init optimizer
      optimizer.zero_grad()

  if (epoch + 1) % 10 == 0:
        print(f'epoch: {epoch+1}, loss = {loss.item(): .4f}')

# # show in image
# predicted = model(feature).detach().numpy()
# plt.show()


epoch: 10, loss =  0.1802
epoch: 20, loss =  0.1789
epoch: 30, loss =  0.1861
epoch: 40, loss =  0.1892
epoch: 50, loss =  0.1926


## save model

In [25]:
FILE = 'hw1_4.pt'

In [26]:
torch.save(model, FILE)

## testing 

In [27]:
ans = torch.empty(len(testing))
for i in range(len(testing)):
  yb = model(testing[i])
  # print(i,yb)
  yb = yb*PM2_5_std + PM2_5_mean
  # print(i,yb)
  ans[i] = yb
  # print(ans)
  # print("-------------------")

In [28]:
ans

tensor([36.0886, 44.4194, 43.3475,  ..., 85.0183, 71.7987, 63.1107],
       grad_fn=<CopySlices>)

## output to csv

In [29]:
import pandas as pd
import os.path
ans_np = ans.detach().numpy() #convert to Numpy array
df = pd.DataFrame(ans_np) #convert to a dataframe


In [30]:
df 

,0
0,36.088577
1,44.419407
2,43.347462
3,40.532475
4,24.354824
...,...
8755,128.046539
8756,106.997726
8757,85.018326
8758,71.798660


In [31]:
df.rename(columns = {0:'PM2.5'}, inplace = True)
df.to_csv(os.path.join('/content/drive/MyDrive/code/python/projects/ADL/ADL_(IM5062)/111ntu-homework1','ans')) #save to file
#df.rename(columns = {:'INDEX'}, inplace = True)


In [32]:
df3 = pd.read_csv('/content/drive/MyDrive/code/python/projects/ADL/ADL_(IM5062)/111ntu-homework1/ans')

In [33]:
df3

,Unnamed: 0,PM2.5
0,0,36.088577
1,1,44.419407
2,2,43.347460
3,3,40.532475
4,4,24.354824
...,...,...
8755,8755,128.046540
8756,8756,106.997730
8757,8757,85.018326
8758,8758,71.798660


In [34]:
df3.rename(columns = {'Unnamed: 0':'INDEX'}, inplace = True)

In [35]:
df3['INDEX'] = df3['INDEX']+1

In [36]:
df3

,INDEX,PM2.5
0,1,36.088577
1,2,44.419407
2,3,43.347460
3,4,40.532475
4,5,24.354824
...,...,...
8755,8756,128.046540
8756,8757,106.997730
8757,8758,85.018326
8758,8759,71.798660


In [37]:
df3.to_csv(os.path.join('/content/drive/MyDrive/code/python/projects/ADL/ADL_(IM5062)/111ntu-homework1','ans.csv'),index=False) #save to file

In [38]:
df4 = pd.read_csv('/content/drive/MyDrive/code/python/projects/ADL/ADL_(IM5062)/111ntu-homework1/ans.csv')

In [39]:
df4 

,INDEX,PM2.5
0,1,36.088577
1,2,44.419407
2,3,43.347460
3,4,40.532475
4,5,24.354824
...,...,...
8755,8756,128.046540
8756,8757,106.997730
8757,8758,85.018326
8758,8759,71.798660


In [40]:
model1 = torch.load(FILE)

In [41]:
ans = torch.empty(len(testing))
for i in range(len(testing)):
  yb = model1(testing[i])
  # print(i,yb)
  yb = yb*PM2_5_std +PM2_5_mean
  # print(i,yb)
  ans[i] = yb
  # print(ans)
  # print("-------------------")

In [42]:
ans

tensor([36.0886, 44.4194, 43.3475,  ..., 85.0183, 71.7987, 63.1107],
       grad_fn=<CopySlices>)

In [43]:
# model.eval()

# print('whole model load')
# for parm in model.parameters():
#     print(parm)